## Градиентный бустинг

$$a_T(x)= \sum_{t=1}^T \alpha_t b_t(x), \quad x\in X, \quad b_t:X\rightarrow \mathbb{R}, \quad\alpha_t\in \mathbb{R}_+$$

**Эвристика:** обучаем $\alpha_t, b_t$ при фиксированных прилылущих.

**Критерий качества** с заданной гладкой функцией потерь $\mathcal{L}(b,y)$:
$$Q(\alpha;b;X^l)=\sum_{i=1}^l\mathcal{L}(\sum_{t=1}^{T-1}\alpha_t b_t(x_i)+\alpha b(x_i), y_i)\rightarrow\underset{\alpha, b}{min}$$
- $(a_{T-1,i})_{i=1}^l$ - вектор текущего приближения
- $(a_{T,i})_{i=1}^l$ - вектор следующего приближения

Градиентный метод минимизации $Q(F)\rightarrow min, f\in \mathbb{R}^l$:
- $a_{0,i}:=$ начальное приближение;
- $a_{T,i}:= a_{T-1,i}-\lambda g_i$ начальное приближение, $i=1,...,l$, где $g_i=\mathcal{L}_f'(a_{T-1,i}, y_i)$ - компоненты вектора градиента, $\lambda$- градиентный шаг;

А добавление одного базового алгоритма: 
$a_{T,i}:= a_{T-1,i}-\lambda b(x_i), \quad i=1,...,l$

Тогда будем искать базовый алгоритм, который приближает вектор антиградиента.

**Алгоритм градиентного бустинга:**

*Вход:* 

- $X^l, Y^l$ - обучающая выборка;
- $T$ - максимальное число базовых алгоритмов;
- $\lambda$ - скорость обучения.

*Выход:*
- базовые алгоритмы и их веса $\alpha_tb_t, \quad t=1,...,T$

*Алгоритм:*
1. инициализируем вектор начальный значений: $a_i=1/l, \quad i=1,..l;$
2. для всех $t=1,...,T:$
- Обучить базовый алгоритм, приближающий антиградиент:  
$b_t:=arg\underset{b\in B}{min}\sum_{i=1}^l(b(x_i)+\mathcal{L}'(w_i,y_i))^2$
- найти вес базового алгоритма $\alpha_t$:  
$\alpha_t:=arg\underset{a>0}{min}\sum_{i=1}^l\mathcal{L}(a_i+\lambda b_t(x_i);y_i)$
- обновить значения:  
$a_i:=a_i+\alpha_t b_t(x_i); \quad i=1,...,l$

**Алгоритм градиентного бустинга адаптация под логарифмическую функцию потерь:**
$$Q(\alpha;b;X^l) = \sum_{i=1}^l log_2(1+e^{-y_i*b(x_i) })\rightarrow \underset{w}{min}$$
Антиградиент: 

$\mathcal{L}'(b(x_i),y_i))= y_i*\sigma(-b(x_i)*y_i)$, где $\sigma(z) = \frac{1}{1+e^{-z}}$

*Вход:* 

- $X^l, Y^l$ - обучающая выборка;
- $T$ - максимальное число базовых алгоритмов;
- $\lambda$ - скорость обучения.

*Выход:*
- базовые алгоритмы и их веса $\alpha_tb_t, \quad t=1,...,T$
- в качестве базовой функции возьмем DecisionTreeRegressor()

*Алгоритм:*
1. инициализируем вектор начальный значений: $a_i=1/l, \quad i=1,..l;$
2. для всех $t=1,...,T:$
- Обучить базовый алгоритм на наблюдениях $\mathcal{L}'(b(x_i),y_i))$;  
- Обновить значения: 
$a_i:=a_i+\lambda b_t(x_i); \quad i=1,...,l$

In [1]:
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.tree import DecisionTreeRegressor
import numpy as np
import pandas as pd

In [2]:
X, y = make_classification(
    n_features=2, n_redundant=0, n_informative=2, random_state=1, n_clusters_per_class=1
)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

datasets = [
    make_moons(noise=0.3, random_state=0),
    make_circles(noise=0.2, factor=0.5, random_state=1),
    linearly_separable,
]

data_moon = datasets[1]

X, y = data_moon

y = np.array([ -1.0 if i == 0 else float(i) for i in y])

## Задание 

1. Реализовать алгоритм градиентного бустинга для логарифмической функции потерь.
2. Оценить качество работы алгоритма на сгенерированных трех наборах данных. 

In [3]:
def sigmoida(z):
    return 1 / (1 + np.exp(-z))

In [4]:
def grad_boost(X, y, T, lambda_):
    l = X.shape[0]
    a = np.ones(l) / l
    b = []
    y[y == 0] = -1
    
    for t in range(T):
        
        # Вычисляем значение антиградиента
        L = y * sigmoida(- a * y)
        
        model = DecisionTreeRegressor()
        
        model.fit(X, L)
        y_pred = model.predict(X)
        
        a += lambda_ * y_pred
        
        b.append(model)
        
    return np.sign(a)

In [5]:
for i, (X, y) in enumerate(datasets):
    a_sign = grad_boost(X, y, 50, 0.1)
    N = np.sum(a_sign[a_sign != y])
    
    print(f'Dataset {i + 1}: {N}')

Dataset 1: 0.0
Dataset 2: 0.0
Dataset 3: 0.0
